# Importing packages

In [1]:
import os

import pandas as pd
import numpy as np

import cv2
import matplotlib.pylab as plt

from glob import glob as glob
from pathlib import Path

In [2]:
crack_prodir = Path("../data/processed/crack_temp")

In [36]:
def analyze_crack_intensity_v3(binary_img, area_thresh_low=0.05, area_thresh_high=0.015, width_thresh=5):

    img_height, img_width = binary_img.shape
    img_area = img_height * img_width

    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    crack_features = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 10:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        bbox = binary_img[y:y+h, x:x+w]
        white_pixels = cv2.countNonZero(bbox)
        # bbox_area = w * h
        crack_ratio = white_pixels / img_area

        # Use minimum of width and height as "local width"
        local_width = min(w, h)

        # Combine area and width to decide intensity
        if crack_ratio < area_thresh_low or local_width < width_thresh:
            intensity = 0  # Low
        elif crack_ratio < area_thresh_high and local_width < width_thresh * 1.5:
            intensity = 1  # Medium
        else:
            intensity = 2  # High

        aspect_ratio = float(w) / h if h != 0 else 0

        print(f"Crack Ratio: {crack_ratio:.4f}, Local Width: {local_width}, Intensity: {intensity}")

        crack_features.append({
            'bbox': (x, y, w, h),
            'area': area,
            'white_pixels': white_pixels,
            'crack_ratio': crack_ratio,
            'aspect_ratio': aspect_ratio,
            'local_width': local_width,
            'intensity': intensity
        })

    return crack_features


In [37]:
def visualize_crack_intensity(img, crack_features):
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    for feat in crack_features:
        x, y, w, h = feat['bbox']
        intensity = feat['intensity']
        green = int(255 * (1 - intensity))
        red = int(255 * intensity)
        color = (0, green, red)
        
        cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
        cv2.putText(output, f"{intensity:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return output


In [ ]:
# binary_img = your_preprocessed_crack_image
crack_images = list(crack_prodir.glob("*.jpg"))[:20]

for img_path in crack_images:
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    
    binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
    features = analyze_crack_intensity_v3(binary)
    vis = visualize_crack_intensity(img, features)
    
    cv2.imshow(f"Crack Intensity - {img_path.name}", vis)
    cv2.waitKey(0)  # Wait for key press before moving to next image

cv2.destroyAllWindows()

Crack Ratio: 0.0920, Local Width: 93, Intensity: 2


Crack Ratio: 0.0776, Local Width: 129, Intensity: 2
Crack Ratio: 0.0845, Local Width: 103, Intensity: 2
Crack Ratio: 0.0736, Local Width: 61, Intensity: 2
Crack Ratio: 0.0579, Local Width: 77, Intensity: 2
Crack Ratio: 0.0654, Local Width: 38, Intensity: 2
Crack Ratio: 0.0975, Local Width: 117, Intensity: 2
Crack Ratio: 0.0664, Local Width: 106, Intensity: 2
Crack Ratio: 0.0784, Local Width: 60, Intensity: 2
Crack Ratio: 0.0682, Local Width: 77, Intensity: 2
Crack Ratio: 0.0037, Local Width: 16, Intensity: 0
Crack Ratio: 0.0498, Local Width: 97, Intensity: 0
Crack Ratio: 0.0870, Local Width: 91, Intensity: 2
Crack Ratio: 0.0682, Local Width: 38, Intensity: 2
Crack Ratio: 0.0595, Local Width: 52, Intensity: 2
Crack Ratio: 0.0575, Local Width: 59, Intensity: 2
Crack Ratio: 0.0658, Local Width: 52, Intensity: 2
Crack Ratio: 0.0576, Local Width: 53, Intensity: 2
Crack Ratio: 0.0827, Local Width: 83, Intensity: 2
Crack Ratio: 0.0269, Local Width: 36, Intensity: 0
Crack Ratio: 0.0250, Local 